<a href="https://colab.research.google.com/github/thiagonobrega/tl-pprl/blob/master/notebooks/GraphAttack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## init

In [ ]:
import sys
import time
import os
import logging

### colab

In [ ]:
!rm -rf tl-pprl/
!git clone -b scale https://github.com/thiagonobrega/tl-pprl.git &> /dev/null
# !pip install --upgrade gdown &> /dev/null
!pip install -r tl-pprl/requirements.txt &> /dev/null

In [ ]:
sys.path.append("./tl-pprl/")
lpath = '.'+os.sep

### getting data

In [ ]:
# https://drive.google.com/drive/folders/16sYLUUCaHy5WIb0jKJgcFxZr-MZ4XsLH?usp=sharing

#books
# url='https://drive.google.com/drive/folders/1W3dosBr5USCrAcn9aomf-tFvPByJOxvm' 
# !gdown --continue --folder --quiet -O priv_eval_tl_pprl/ {url}

# url='https://drive.google.com/drive/folders/16sYLUUCaHy5WIb0jKJgcFxZr-MZ4XsLH'
# #zip
# url = 'https://drive.google.com/uc?id=16QmWGrKpN4o_95rbJ6I0Iy7bDZYj6RCx'
# !gdown --continue --folder --quiet {url}
#54mb
# !gdown --continue --folder {url}

# https://1drv.ms/u/s!AiduPqZxUF_qla8eaYkkqSAnIy--ig?e=keMfpi
!curl 'https://1ccheq.bn.files.1drv.com/y4mVyu9oQbrpQLTZKEf-IzasRDsIZNUUijKD438tKW3YJiCOdT6D2Vb9oVCXTunKRaMBQDxOI8xYIxjbHu8SKo4B3LydiA-Y4f3D2LvNPz8hG4Q-teW9GPmGNs09Dz3-2W_VE7GYtmVEkizPMsx5n6ZDY22PYwWXdxXxploAnWm_fRgUMAhCWVbUMxfEeZish41SW7OMIOy5RqDBZajwQqaGw' \
  -H 'authority: 1ccheq.bn.files.1drv.com' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' \
  -H 'accept-language: pt-BR,pt;q=0.9' \
  -H 'referer: https://onedrive.live.com/' \
  -H 'sec-fetch-dest: iframe' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-site: cross-site' \
  -H 'sec-gpc: 1' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36' \
  --compressed --output datasets.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.1M  100 45.1M    0     0  28.0M      0  0:00:01  0:00:01 --:--:-- 28.0M


### extracting data

In [ ]:
!rm -rf datasets

In [ ]:
!unzip datasets.zip -d datasets

Archive:  datasets.zip
   creating: datasets/books/
  inflating: datasets/books/processed_amazon-barnesnobel-small.zip  
  inflating: datasets/books/processed_amazon-barnesnobel.zip  
  inflating: datasets/books/processed_goodreads-barnesnobel.zip  
   creating: datasets/census/
  inflating: datasets/census/processed_census.zip  
   creating: datasets/dblp-acm/
 extracting: datasets/dblp-acm/processed_DBLP-ACM.zip  
   creating: datasets/movies/
  inflating: datasets/movies/processed_imdb-rottentomatos.zip  
  inflating: datasets/movies/processed_imdb-tmd.zip  
   creating: datasets/mvr/
 extracting: datasets/mvr/michiganvoters_100000_0.1.zip  
 extracting: datasets/mvr/michiganvoters_10000_0.1.zip  
  inflating: datasets/mvr/michiganvoters_2000_0.1.zip  
 extracting: datasets/mvr/michiganvoters_500000_0.1.zip  
   creating: datasets/ncvr/
   creating: datasets/restaurants/
  inflating: datasets/restaurants/processed_fodors-zagats.zip  
  inflating: datasets/restaurants/processed_yelp-

### Selecting data

In [ ]:
!rm datasets/mvr/michiganvoters_500000_0.1.zip
!rm datasets/mvr/michiganvoters_100000_0.1.zip
# !rm datasets/mvr/michiganvoters_10000_0.1.zip
!rm datasets/mvr/michiganvoters_2000_0.1.zip

In [ ]:
!rm -rf datasets/ncvr/priv_ncvoters_100000_0.1.zip
!rm -rf datasets/ncvr/priv_ncvoters_50000_0.1.zip
!rm -rf datasets/ncvr/priv_ncvoters_20000_0.1.zip  
!rm -rf datasets/ncvr/priv_ncvoters_10000_0.1.zip  

In [ ]:
!mkdir saida
!mkdir saida/graphs
!mkdir saida/regre-models
# !mv priv_eval_tl_pprl datasets

### mount gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### windows local

In [ ]:
# lpath = 'd:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep+'workspace'+os.sep+'tl@pprl'+os.sep
# sys.path.append(lpath)

# run

## setup geral

In [ ]:
from graph_attack.graph_attack import load_data_set, gen_q_gram_sets, genG, step04
from graph_attack.utils import simcalc
from graph_attack.utils.graph import SimGraph
from graph_attack.graph_attack import encode_ds

### Dataset

In [ ]:
# plain_attr_list = [1,3,2] #census
encode_base_data_set_name='ncvr'
plain_attr_list= [2,3,1] #mvr
plain_attr_list= [2,1]

plain_base_data_set_name='ncvr'
encode_attr_list = [2,3,1]
encode_attr_list = [2,1]
#MVR
# plain_base_data_set_name='ncvr'
plain_base_data_set_name='mvr'
#Encoded file:    ['first_name', 'middle_name', 'last_name']
encode_attr_list = [2,3,4]
encode_attr_list = [2,4]

# bf_enc_param = [0.3,0.3,0.4]
bf_enc_param = [0.5,0.5]

#similarity function
plain_sim_funct_name='jacc'
encode_sim_funct_name='jacc'


# anonymization
q=2
padded_flag=False #sem pad
# PAD_CHAR = chr(1) 
# bf
encode_method='bf'
bf_hash_type='rh'
bf_num_hash_funct='opt'
bf_len=200
# bf_encode='clk' #rbf
bf_encode='rbf'
bf_harden='none'


assert plain_sim_funct_name in ['dice','jacc','hamm']
assert encode_sim_funct_name in ['dice','jacc','hamm']

# Define encode and plaintext blocking methods
#
encode_blck_method = 'minhash' # hmlsh, minhash, soundex, none
plain_blck_method  = 'minhash' # minhash, soundex, none
# plain_blck_method = 'none'
# plain_blck_method = 'minhash'
# plain_blck_method = 'minhash'


##
# atack param
sim_diff_adjust_flag=True
min_sim = .05

### para ajuste das similaridades
if(sim_diff_adjust_flag == True):
  # Define the regression model
  if(encode_method == '2sh'):
    regre_model_str = 'poly' # linear, poly, isotonic
  else:
    regre_model_str = 'linear' # linear, poly, isotonic
else: 
  regre_model_str = 'none'


### similarity function

In [ ]:
# Initialise the actual similarity functions to be used
#plain
if (plain_sim_funct_name == 'dice'):
    plain_sim_funct = simcalc.q_gram_dice_sim
elif (plain_sim_funct_name == 'jacc'):
  plain_sim_funct = simcalc.q_gram_jacc_sim
else:
  raise Exception('This should not happen: sim function not allowed')

#encoded
if(encode_method == '2sh'):
  if (encode_sim_funct_name == 'dice'):
    encode_sim_funct = simcalc.q_gram_dice_sim
  elif (encode_sim_funct_name == 'jacc'):
    encode_sim_funct = simcalc.q_gram_jacc_sim
  else:
    raise Exception('This should not happen: hamming is not allowed to 2sh') # nao faz sentido
else:
  if (encode_sim_funct_name == 'dice'):
    encode_sim_funct = simcalc.bit_array_dice_sim
  elif (encode_sim_funct_name == 'hamm'):
    encode_sim_funct = simcalc.bit_array_hamm_sim
  elif (encode_sim_funct_name == 'jacc'):
    encode_sim_funct = simcalc.bit_array_jacc_sim
  else:
    raise Exception('This should not happen: sim function not defined!')

### other variables

In [ ]:
if (encode_method == 'bf'):
  encode_method_str = 'bf-%s-%s-%d-%s-%s' % (bf_hash_type, bf_num_hash_funct,
                      bf_len, bf_encode, bf_harden)
elif (encode_method == 'tmh'):
  encode_method_str = 'tmh-%d-%s-%d-%d-%d' % (tmh_num_hash_bits,
                      tmh_hash_funct, tmh_num_tables, tmh_key_len, tmh_val_len)
elif (encode_method == '2sh'):
  encode_method_str = '2sh-%d-%d-%d' % (cmh_max_rand_int,
                                        cmh_num_hash_funct, cmh_num_hash_col)

In [ ]:
today_str = time.strftime("%Y%m%d", time.localtime())
now_str =   time.strftime("%H%M", time.localtime())
today_time_str = time.strftime("%Y%m%d %H:%M:%S", time.localtime())

In [ ]:
graph_path = 'graphs/'
random_seed=101
# bf_enc_param='clk'
# bf_enc_param='rbf'

generated_graph_flag = True  # Graphs were generated in this run
graph_path = lpath + 'saida' + os.sep
reg_path = lpath + 'saida' + os.sep

encode_graph_file_name = graph_path + 'encode_graph_file_name'
plain_graph_file_name  = graph_path + 'plain_graph_file_name'
bf_num_hash_funct,bf_hash_type
# ['linear','isotonic','poly',]
regre_model_str='linear'

### step 03 variable

In [ ]:
regre_model_str='linear'

# Get the overall minimum similarity for the graphs
#
min_sim = 1.0


sim_diff_interval_size = 0.05

### Srep 04 variables

In [ ]:
#variaveis importantes
# qg_graph_node_id_dict = {}
# ba_graph_node_id_dict = {}

# qg_graph_node_id_dict = {}
# ba_graph_node_id_dict = {}

# QG_sim_graph, BA_sim_graph, plain_graph_num_node, encode_graph_num_node, attck_res_header_list, attck_res_val_list

### graph paramns

# sim_hash_block_funct_list = ['all_in_one'] # 'all_in_one','hlsh'
# sim_comp_funct_list = ['allpairs'] # 'allpairs', 'simtol'

# graph_sim_list = [('[0.2]',[0.2])]
graph_sim_list = [('[0.2]',[0.2])] #('[0.9-0.5]',[0.9,0.7,0.5]),
                                     #('[0.5]',[0.5])

graph_node_min_degr_list = [('5',5)]

sim_hash_num_bit_list = [('1000',1000)]
sim_hash_match_sim = 0.9
sim_hash_block_funct_list = ['hlsh']
sim_comp_funct_list = ['simtol']

graph_feat_list_list = [('all',['node_freq','max_sim','min_sim','degree',
                            'degree_histo1','degree_histo2','sim_avr',
                            'sim_std','egonet_degree','egonet_density',
                            #'pagerank', # very time consuming
                            'between_central',
                            #'closeness_central', # very time consuming
                            'degree_central',
                            #'eigenvec_central', # very time consuming
                            ]),
                            
                     ('no-histo',['node_freq','max_sim','min_sim','degree',
                                   'sim_avr', 'sim_std','egonet_degree',
                                   'egonet_density','between_central',
                                   'degree_central', 'eigenvec_central',
                            ]),
                     
                     ('one-central',['node_freq','max_sim','min_sim','degree',
                            'degree_histo1', 'degree_histo2', 'sim_avr',
                            'sim_std','egonet_degree', 'egonet_density',
                            'degree_central',
                            ]),
                     
                     ('no-nf-central',['max_sim','min_sim','degree',
                                          'degree_histo1','degree_histo2',
                                          'sim_avr','sim_std',
                                          'egonet_degree','egonet_density']),
                     
                     ('no-central',['node_freq','max_sim','min_sim',
                                   'degree','degree_histo1',
                                   'degree_histo2','sim_avr','sim_std',
                                   'egonet_degree','egonet_density']),
                     
                     ('node2vec', ['node2vec']),
]

graph_feat_select_list = [('all','all'),
                          #('non-zero','nonzero'),
                          #'top-10':10,
                          #('top-10',10)
                          #('min-std-0.4',0.4),
                         ]

# For the 'simtol' comparison approach set the similarity tolerance values:
# lower: how much lower the encoded similarity can be,
# upper: how much higher the encoded similarity can be
#
bf_hash_sim_low_tol =  0.01  # If the encoding are Bloom filters
bf_hash_sim_up_tol =   0.25

tmh_hash_sim_low_tol =  0.05  # If the encoding are tabulation min hashes
tmh_hash_sim_up_tol =   0.05

cmh_hash_sim_low_tol =  0.05  # If the encoding are tabulation min hashes
cmh_hash_sim_up_tol =   0.01

adj_hash_sim_low_tol =  0.05  # If the encoded similarities have been adjusted
adj_hash_sim_up_tol =   0.05

#ajustes
anon_hash_sim_low_tol = bf_hash_sim_low_tol
anon_hash_sim_up_tol = bf_hash_sim_up_tol


hash_sim_min_tol = 0.05  # Also for the 'simtol' comparison approach, how much
                         # tolerance to count the number of edges

# Numpy and Scipy Cosine give the same results and are always best - so only
# use those
#
#final_sim_funct_list = ['lsh_cosine','cosine_scipy','cosine_numpy',
#                        'euclidean']
#final_sim_funct_list = ['edge_sim_diff']

# The parameters for the Hamming LSH blocking of the similarity hashes in the
# graph matching step
#
# if('ncvoter' in encode_data_set_name or 'ncvr' in encode_data_set_name):
graph_block_hlsh_num_sample =      20
graph_block_hlsh_rel_sample_size = 10  # Divide bit array length by this

# else:
#   graph_block_hlsh_num_sample =      50
#   graph_block_hlsh_rel_sample_size = 50  # Divide bit array length by this

# Also get a sorted list of all similarities over all parameter settings
#
all_sim_set = set()

for (sim_list_name, sim_list) in graph_sim_list:
  min_sim = min(min_sim, min(sim_list))
  all_sim_set = all_sim_set | set(sim_list)

all_sim_list = sorted(all_sim_set)


## Exec: Load data

In [ ]:
from zipfile import ZipFile


def extract_ds(ed,filename,target='a.csv'):
  with ZipFile(ed+filename, 'r') as zipObj:
    listOfFileNames = zipObj.namelist()
    for fileName in listOfFileNames:
        if fileName == target:
            zipObj.extract(fileName, ed)

In [ ]:
encoded_dir = 'datasets'+os.sep+encode_base_data_set_name+os.sep
plain_dir = 'datasets'+os.sep+plain_base_data_set_name+os.sep

enc_files = [f for f in os.listdir(encoded_dir) if '0.1.zip' in f]
plain_files = [f for f in os.listdir(plain_dir) if '.zip' in f]

In [ ]:
# formatter = logging.Formatter('%(asctime)s %(levelname)s: %(funcName)s:%(lineno)d %(message)s')
formatter = logging.Formatter('%(funcName)s:%(lineno)d %(message)s')

# create logger with 'spam_application'
logger = logging.getLogger()
logger.handlers.pop(0)
# logger.handlers

logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('exec_100k.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)

# just so we can see things as they happen in stdout/stderr
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
ch.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

### run

In [ ]:
#itera nos dados
# for plain_file in plain_files[:1]:
for plain_file in plain_files:
  #extrai os dados
  extract_ds(plain_dir,plain_file)
  # le os dados planos
  plain_rec_attr_val_dict, plain_attr_name_list, plain_num_rec_loaded, plain_soundex_val_dict =\
      load_data_set(lpath + plain_dir+'a.csv',plain_attr_list,0,[2], #TODO:jogar esses parametros p cima
                      header_line_flag=True,
                      col_sep_char=';')
  #removendo os arquivos apos o experimento
  os.remove(plain_dir+'a.csv')

  # for enc_file in enc_files[:1]:
  for enc_file in enc_files:
    #extrai os dados
    print('')
    print(plain_file,enc_file)
    print('')
    extract_ds(encoded_dir,enc_file,target='b.csv')
    #le os dados codificados
    encode_rec_attr_val_dict, encode_attr_name_list,encode_num_rec_loaded, encode_soundex_val_dict =\
        load_data_set(lpath +encoded_dir+'b.csv',encode_attr_list,0,[2],#TODO:jogar esses parametros p cima
                        header_line_flag=True,
                        col_sep_char=';')
    # encode_rec_attr_val_dict, encode_attr_name_list,encode_num_rec_loaded, encode_soundex_val_dict =\
    # load_data_set(lpath + plain_dir+'b.csv',plain_attr_list,0,[2], #TODO:jogar esses parametros p cima
    #                   header_line_flag=True,
    #                   col_sep_char=';')

    #removendo os arquivos apos o experimento
    # os.remove(encoded_dir+'a.csv')

    if (encode_attr_name_list != plain_attr_name_list):
      print('*** Warning: Different attributes used in encoded and ' + \
            'plain-text files:')
      print('***   Encoded file:   ', encode_attr_name_list)
      print('***   Plain-text file:', plain_attr_name_list)
      # encode_attr_name_list = plain_attr_name_list

    # Generate q-gram sets for records
    #
    encode_q_gram_dict = gen_q_gram_sets(encode_rec_attr_val_dict, q, padded_flag)
    plain_q_gram_dict =  gen_q_gram_sets(plain_rec_attr_val_dict, q, padded_flag)

    enc_id_set = set(encode_q_gram_dict.keys())
    plain_id_set = set(plain_q_gram_dict.keys())

    common_rec_id_set = enc_id_set.intersection(plain_id_set)

    print('The two datasets have %d overlapping records (%.1f %%)' % \
          (len(common_rec_id_set), 200*float(len(common_rec_id_set))/(len(enc_id_set)+len(plain_id_set)))
    )

    ###########
    ########### Step 02
    ###########
    
    # Create the graph pickle file names, and if they both are available then
    # load the graphs from files
    #
    plain_attr_list_str =  str(plain_attr_list).replace(', ','_')[1:-1]
    encode_attr_list_str = str(encode_attr_list).replace(', ','_')[1:-1]

    # The generated graph file names
    #          
    encode_str = 'encode-sim-graph-%s-%s-%d-%d-%s-%s-%s-%s-%.3f-%s' \
                % (encode_base_data_set_name, encode_attr_list_str,
                    encode_num_rec_loaded, q, str(padded_flag).lower(),
                    regre_model_str.lower(),
                    encode_sim_funct_name, encode_blck_method, min_sim,
                    encode_method_str)
                
    plain_str = 'plain-sim-graph-%s-%s-%d-%d-%s-%s-%s-%s-%.3f-%s' % \
                (plain_base_data_set_name, plain_attr_list_str,
                  plain_num_rec_loaded, q, str(padded_flag).lower(),
                  regre_model_str.lower(),
                  plain_sim_funct_name, plain_blck_method, min_sim,
                  encode_method_str)

    encode_graph_file_name = encode_str + '.pickle'
    plain_graph_file_name = plain_str + '.pickle'


    encode_graph_file_name = graph_path + encode_graph_file_name
    plain_graph_file_name  = graph_path + plain_graph_file_name

    ## encode dataset
    print('**** Step 2 :: Encoding Dataset')
    e = encode_ds(encode_q_gram_dict,encode_attr_list,encode_rec_attr_val_dict,
                plain_rec_attr_val_dict,
                encode_method,bf_len,q,bf_encode,
                bf_hash_type=bf_hash_type,
                bf_enc_param=bf_enc_param,
                bf_num_hash_funct=bf_num_hash_funct,
                random_seed=random_seed, # definir
            )
    
    encode_hash_dict,hashing_time,plain_num_ent,encode_num_ent,common_num_ent = e[0],e[1],e[2],e[3],e[4]
    print('**** Step 2 :: Done')
    
    
    ###############
    ############### STEP 03
    ###############
    print('**** Step 3 :: Generating Graph')
    try:
      z = genG(plain_num_ent,encode_num_ent,
          encode_method, # encoding
          # encode
          encode_sim_funct_name,
          encode_hash_dict,
          encode_q_gram_dict,
          encode_rec_attr_val_dict,
          encode_blck_method, #'hmlsh': Hamming LSH blocking 'minhash' :  Min-hash LSH blocking
          #plain
          plain_q_gram_dict,
          plain_rec_attr_val_dict,
          plain_sim_funct_name,
          plain_blck_method,
          #soundex
          encode_soundex_val_dict,
          plain_soundex_val_dict,
          # encoded_plain_vars
          # regression
          regre_model_str,
          plain_base_data_set_name,
          plain_attr_list_str,
          q,
          padded_flag,
          encode_method_str,
          encode_base_data_set_name,plain_num_rec_loaded,encode_attr_list_str,encode_num_rec_loaded,
          #outros
          min_sim, # vem la de cima
          all_sim_list,
          #
          num_samples = 20000, # regression num samples
          random_seed=101,
          ## anonimizacao
          bf_hash_type=bf_hash_type,
          bf_num_hash_funct=bf_num_hash_funct,
          bf_len=bf_len,
          bf_encode=bf_encode,
          bf_harden='None',
          # leitura dos grafos
          graph_path = graph_path,
          regre_file_path = reg_path,
          plain_graph_file_name=plain_graph_file_name,
          encode_graph_file_name=encode_graph_file_name,
          # utilizar apenas atributos em comum
          include_only_common = False,
          common_rec_id_set={},#setar caso seja utilizado
          #nao sei o que e isso mas estava hardoced
          same_ba_blck = False,
          #ajueste de similaridade
          sim_diff_interval_size = sim_diff_interval_size,
          sim_diff_adjust_flag=sim_diff_adjust_flag
        ) 
      
      # repassando variaveis
      QG_sim_graph, BA_sim_graph, plain_graph_num_node, encode_graph_num_node,\
      attck_res_header_list, attck_res_val_list = z
      print('**** Step 3 :: Done')

      #########
      ######### Step 04 and 05
      #########

      exp_params = [ plain_attr_list_str,plain_num_rec_loaded,encode_attr_list_str,\
        encode_num_rec_loaded,padded_flag,q,plain_sim_funct_name,\
        encode_sim_funct_name,encode_method_str,encode_blck_method,\
        plain_blck_method
      ]
      
      p = plain_file.split('.')[0]
      e = enc_file.split('.')[0]
      ataque_file = 'ataque-'+p+'-'+e+'.csv'

      print('**** Step 4 :: Starting...')

      step04(QG_sim_graph,BA_sim_graph,
          graph_node_min_degr_list,
          min_sim,
          all_sim_list,
          plain_rec_attr_val_dict,
          encode_rec_attr_val_dict,
          plain_q_gram_dict,
          encode_q_gram_dict,
          #
          sim_comp_funct_list,
          sim_hash_num_bit_list,
          sim_hash_block_funct_list,
          # sim_hash_match_sim,
          #
          hash_sim_min_tol,
          adj_hash_sim_low_tol,
          adj_hash_sim_up_tol,
          anon_hash_sim_low_tol,
          anon_hash_sim_up_tol,
          #
          plain_str,
          encode_str,
          #
          graph_sim_list,
          graph_feat_list_list,
          graph_feat_select_list,
          #
          plain_base_data_set_name,
          encode_base_data_set_name,
          #
          encode_method, 
          #
          attck_res_header_list, attck_res_val_list,
          #
          sim_hash_match_sim=sim_hash_match_sim,
          # blocagem
          graph_block_hlsh_rel_sample_size=graph_block_hlsh_rel_sample_size,
          graph_block_hlsh_num_sample=graph_block_hlsh_num_sample,
          #
          plain_graph_num_node=0,
          encode_graph_num_node=0,
          # attack_res_file_name='ataque.csv',
          attack_res_file_name=ataque_file,
          random_seed=random_seed,
          feat_path = 'saida' + os.sep + 'feats' + os.sep,
          sim_diff_adjust_flag=sim_diff_adjust_flag,
          exp_params=exp_params,
      ) #fim do step 04
      # copiar aqui
      print('**** Step 4 :: Done')
      !cp *.csv '/content/drive/MyDrive/Colab Notebooks/'
    # except ValueError:
    except KeyError:
      # ValueError: zero-size array to reduction operation minimum which has no identity
      # investigar esse erro
      pass


michiganvoters_10000_0.1.zip priv_ncvoters_100000_0.1.zip

*** Warning: Different attributes used in encoded and plain-text files:
***   Encoded file:    ['first_name', 'last_name']
***   Plain-text file: ['firstname', 'lastname']
The two datasets have 0 overlapping records (0.0 %)
**** Step 2 :: Encoding Dataset
**** Step 2 :: Done
**** Step 3 :: Generating Graph
**** Step 3 :: Done
**** Step 4 :: Starting...


In [ ]:
!cp *.log '/content/drive/MyDrive/Colab Notebooks/'
!cp *.csv '/content/drive/MyDrive/Colab Notebooks/'

ab

## nao rodar

In [ ]:
print(ent_id)
print(node_key_val)

print(encode_key_q_gram_key_dict[node_key_val])
print(tuple(sorted(q_gram_set)))

encode_q_gram_dict['cw1297608']
'bf38941'
lflag = node_key_val

cw1297608
bitarray('111000110010110110000000100000000100100101000001000010100100100011101011010001000010001011001100010100000010000011100000')
('ei', 'en', 'er', 'fe', 'gh', 'if', 'ig', 'je', 'le', 'ni', 'nn')
('ei', 'en', 'er', 'fe', 'gh', 'if', 'ig', 'je', 'le', 'ni')


In [ ]:
# First add all records as nodes to the two graphs, and also generate two
# dictionaries where keys are the values added as nodes into the graphs while
# values are the original sets or encodings (that cannot be used as nodes)
# This ensures we compare each unique pair for q-grams / bit-arrays only once
#
plain_node_val_dict =  {}
encode_node_val_dict = {}

# A dictionary where keys are node key values based on q-grams (i.e.
# plain-text values) and values are their corresponding bit arrays (these
# are the true matching q-gram sets to bit arrays to be used for the
# similarity difference calculations)
#
encode_plain_node_val_dict = {}
encode_key_q_gram_key_dict = {}

for (ent_id, bit_array) in encode_hash_dict.items():
  if ent_id == 'cw1297608':
    print(1)
  q_gram_set = encode_q_gram_dict[ent_id]

  if(encode_method == '2sh'):
    node_key_val = [str(val) for val in bit_array]
    node_key_val = tuple(sorted(node_key_val))  # Sets cannot be dictionary keys
  else:
    node_key_val = str(bit_array)  # Bit arrays cannot be dictionary keys  

  if lflag == node_key_val:
    print(ent_id)

  encode_node_val_dict[node_key_val] = (q_gram_set, bit_array)

  if(node_key_val in encode_key_q_gram_key_dict):
    assert encode_key_q_gram_key_dict[node_key_val] == tuple(sorted(q_gram_set))
  else:
    encode_key_q_gram_key_dict[node_key_val] = tuple(sorted(q_gram_set))


bf38941
1
cw1297608


AssertionError: ignored

## Clean

In [ ]:
!rm *.csv
!rm *.log
!rm *.png
!rm saida/*.pickle
!rm saida/*.sav
!rm saida/feats/*